<a href="https://colab.research.google.com/github/AlbertoRoldanSastre/AlbertoRoldanSastre/blob/main/efficientdet_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Efficientdet tensorflow

https://github.com/tensorflow/models

# PASO 00. DRIVE


In [ ]:
# 1. Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# PASO 0: DOWNLOAD DE DATASET:

Opción 1:
Descargar en COCO.JSON y pasar a tensorflow TFrecord con el script:

Descargar en COCO JSON y luego usar el script create_coco_tfrecord.py (en datasets/ del repo) para convertir a TFRecord.

Antes que todo eso, pasas a bounding box, haces tiling, oversamplimg , augmentation y ya con el último COCO.JSON, a TFRecord



Repite algo parecido para validación y test, cambiando las rutas.


Intalar un par de cosas antes de transfromar a tfrecord:

Opción 2:
El repo oficial está pensado para leer TFRecord con las anotaciones embebidas.

Roboflow a veces te ofrece descargar directamente en TFRecord. Si ese export funciona bien, genial: ya tienes el dataset listo para “main.py” en EfficientDet.

**PROBLEMA**
EN TFRecord desde Roboflow solo guarda las bounding boxes.
Es mejor seguir todo igual hasta tf record como si fuera pytorch

# PASO 1: Preparar Entorno y Clonar el Repo

In [ ]:
!pip uninstall -y tensorflow tensorflow-text tensorflow-estimator

In [ ]:
!pip install tensorflow==2.15.0 tensorflow-estimator==2.15.0 tensorflow_text==2.15.0 tensorflow-io


verificar versiones tras reinciiar:

empezar

In [ ]:
# Clonar el repo
!git clone --depth 1 https://github.com/tensorflow/models.git


Cloning into 'models'...
remote: Enumerating objects: 4321, done.
remote: Counting objects: 100% (4321/4321), done.
remote: Compressing objects: 100% (3338/3338), done.
remote: Total 4321 (delta 1208), reused 2043 (delta 910), pack-reused 0 (from 0)
Receiving objects: 100% (4321/4321), 53.31 MiB | 21.92 MiB/s, done.
Resolving deltas: 100% (1208/1208), done.


In [ ]:
#Compilar los archivos .proto
%cd models/research
!protoc object_detection/protos/*.proto --python_out=.

/content/models/research


In [ ]:
import os
import sys

# Agrega a PYTHONPATH los directorios necesarios
os.environ['PYTHONPATH'] = "/content/models/research:/content/models/research/slim:" + os.environ.get('PYTHONPATH', '')
sys.path.append('/content/models/research')
sys.path.append('/content/models/research/slim')

# Opcional: Verifica que las rutas se han añadido
print("PYTHONPATH:", os.environ['PYTHONPATH'])
print("sys.path:", sys.path)

PYTHONPATH: /content/models/research:/content/models/research/slim:/env/python
sys.path: ['/content', '/env/python', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.11/dist-packages/IPython/extensions', '/root/.ipython', '/content/models/research', '/content/models/research/slim']


In [ ]:
!ls -lh /content/models/research/object_detection

total 732K
drwxr-xr-x 2 root root 4.0K Mar 17 09:00 anchor_generators
drwxr-xr-x 2 root root 4.0K Mar 17 09:00 box_coders
drwxr-xr-x 2 root root 4.0K Mar 17 09:00 builders
drwxr-xr-x 2 root root 4.0K Mar 17 09:00 colab_tutorials
drwxr-xr-x 4 root root 4.0K Mar 17 09:00 configs
-rw-r--r-- 1 root root  765 Mar 17 09:00 CONTRIBUTING.md
drwxr-xr-x 2 root root 4.0K Mar 17 09:00 core
drwxr-xr-x 2 root root 4.0K Mar 17 09:00 data
drwxr-xr-x 2 root root 4.0K Mar 17 09:00 data_decoders
drwxr-xr-x 4 root root 4.0K Mar 17 09:00 dataset_tools
drwxr-xr-x 6 root root 4.0K Mar 17 09:00 dockerfiles
-rw-r--r-- 1 root root  55K Mar 17 09:00 eval_util.py
-rw-r--r-- 1 root root  20K Mar 17 09:00 eval_util_test.py
-rw-r--r-- 1 root root  16K Mar 17 09:00 exporter_lib_tf2_test.py
-rw-r--r-- 1 root root  14K Mar 17 09:00 exporter_lib_v2.py
-rw-r--r-- 1 root root 7.4K Mar 17 09:00 exporter_main_v2.py
-rw-r--r-- 1 root root  28K Mar 17 09:00 exporter.py
-rw-r--r-- 1 root root  58K Mar 17 09:00 exporter_tf1_tes

In [ ]:
!pip install lvis

# para convertir json a tfrecord:

In [ ]:
!pip install contextlib2

In [ ]:
!pip install protobuf==3.20.3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-language 2.16.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 3.20.1 which is incompatible.
google-cloud-pubsub 2.28.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 3.20.1 which is incompatible.
google-cloud-bigtable 2.29.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 3.20.1 which is incompatible.
google-cloud-resource-manager 1.14.1 requires protobuf!=4.21.0,!=4.2

ste mensaje indica un conflicto de dependencias: algunas librerías (por ejemplo, tensorflow-metadata, grpcio-status y otros paquetes de Google) requieren una versión de protobuf mucho más reciente (≥4.25.2 o ≥4.21.6) para Python 3.11, mientras que para que funcione correctamente la API de detección de TensorFlow (y el código generado a partir de los archivos .proto) se recomienda usar una versión más antigua de protobuf (por ejemplo, 3.20.3).

¿Es imprescindible para lo que deseas?
Sí e imprescindible:
Todas las dependencias deben ser compatibles para que tu entorno de entrenamiento funcione correctamente. Si hay conflictos, podrías experimentar errores en funciones clave o incluso fallos en la ejecución.
Opciones para resolverlo:
Cambiar la versión de Python en Colab:
La forma más robusta es usar un entorno con Python 3.9 o 3.10. En Colab, esto no es trivial (ya que la imagen predeterminada es Python 3.11), pero puedes considerar utilizar un entorno local o buscar una imagen de Colab que use Python 3.10. Con Python 3.9/3.10, podrías instalar versiones de TensorFlow y protobuf compatibles con todos los paquetes.

Actualizar todas las dependencias (no recomendable):
Intentar forzar el uso de una versión de protobuf más alta (por ejemplo, ≥4.25.2) podría romper la compatibilidad con el código generado por protoc en la API de detección. La API está escrita y probada con versiones antiguas de protobuf, por lo que actualizar puede generar errores difíciles de resolver.

Usar un entorno virtual distinto:
Si tienes la posibilidad de configurar un entorno (por ejemplo, localmente o en un contenedor) con Python 3.9, podrías instalar TensorFlow 2.15 o 2.18 junto con la versión de protobuf requerida (por ejemplo, 3.20.3 o versiones compatibles) sin conflictos con tensorflow-metadata y demás.

Recomendación:
Para el desarrollo de tu tesis y para que el entorno sea estable, te recomiendo usar un entorno con Python 3.9 o 3.10. Esto permitirá que instales versiones de TensorFlow y las dependencias (incluyendo tensorflow-metadata, grpcio-status, etc.) que sean compatibles entre sí. En Colab, esto puede ser complicado, por lo que si decides usar Colab, quizás debas considerar ejecutar el entrenamiento en un entorno local o en una máquina virtual configurada con una versión de Python compatible.

Resumen:
El conflicto surge porque algunas librerías requieren una versión de protobuf ≥4.25.2, mientras que la API de detección de TensorFlow (y el código generado) es compatible con una versión antigua (3.20.3).
Es esencial resolver estos conflictos para que tu modelo se entrene de forma estable.
La solución más robusta es usar un entorno con Python 3.9/3.10, en el cual puedas instalar versiones de todas las dependencias sin conflicto.
Si no puedes cambiar la versión de Python en Colab, es posible que tengas que buscar una solución intermedia, pero ten en cuenta que forzar una versión más reciente de protobuf puede romper la compatibilidad con la API de detección.
Espero que esta explicación te ayude a entender el conflicto y te oriente sobre la mejor manera de resolverlo para tu proyecto. Si tienes más dudas o necesitas más detalles, ¡aquí estoy para ayudarte!

In [ ]:
import google.protobuf
print(google.protobuf.__version__)

3.20.1


In [ ]:
!python models/research/object_detection/dataset_tools/create_coco_tf_record.py \
    --logtostderr \
    --train_image_dir="/content/drive/MyDrive/prueba_efficient/prueba/train" \
    --val_image_dir="/content/drive/MyDrive/prueba_efficient/prueba/valid" \
    --test_image_dir="/content/drive/MyDrive/prueba_efficient/prueba/test" \
    --train_annotations_file="/content/drive/MyDrive/prueba_efficient/prueba/annotations/instances_train.json" \
    --val_annotations_file="/content/drive/MyDrive/prueba_efficient/prueba/annotations/instances_valid.json" \
    --testdev_annotations_file="/content/drive/MyDrive/prueba_efficient/prueba/annotations/instances_test.json" \
    --output_dir="/content/COCO_TFRecords"

In [ ]:
# Define la carpeta de destino en Google Drive
dest_folder = "/content/drive/MyDrive/tensorflow_prueba"
!mkdir -p "{dest_folder}"                                                           #El comando mkdir -p crea esa carpeta (y cualquier subcarpeta necesaria) si no existe.

# Copia la carpeta de resultados (ajusta el nombre según el que se haya generado)
!cp -r /content/COCO_TFRecords "{dest_folder}/"   #copiar los reultados del modelo en drive


# Comprueba que se copiaron los archivos (opcional)
!ls "{dest_folder}"                                                                 #verifica que los archivos se han copiado correctamente

# Opcional: comprimir la carpeta de resultados para descargarla a tu PC
#!zip -r /content/saved/prueba.zip "{dest_folder}/efficient_prueba"   #Aquí se comprime la carpeta que acabas de copiar en Drive (la subcarpeta focas_training_prueba1_640_24_02_25 dentro de foquitas) en un archivo ZIP llamado foquitas.zip que se guarda en /content (la carpeta principal del entorno de Colab).

# Descargar el archivo ZIP a tu PC
#from google.colab import files
#files.download('/content/inferencia_2.zip')

buscar archivo:
en este caso , el d1 que tienen entrada imagenes 640, pero hay más
buscar en carpeta

/content/models/research/object_detection/configs/tf2/

In [ ]:
!find /content/models/research/object_detection/configs/tf2/ -type f -name ssd_efficientdet_d1_640x640_coco17_tpu-8.config

Antes de entrenar debemos cambiar esos archivos de la siguiente forma:

los input_paths son las rutas a donde estan los tf record, poniendo al final el nombre gneral del archivo que detecta train, o valid, en su parte correspondiente

y label_map es el archivo pbtxt donde pones todas tus clases , hay un ejemplo en carpeta tensorflow mia

In [ ]:
train_input_reader {
  input_path: "/content/drive/MyDrive/tensorflow_prueba/COCO_TFRecords/coco_train.record"
  label_map_path: "/content/drive/MyDrive/tensorflow_prueba/label_map.pbtxt"
  shuffle: true
  num_readers: 1
}

eval_input_reader {
  input_path: "/content/drive/MyDrive/tensorflow_prueba/COCO_TFRecords/coco_val.record"
  label_map_path: "/content/drive/MyDrive/tensorflow_prueba/label_map.pbtxt"
  shuffle: false
  num_readers: 1
}

# Train:


el mode_dir= es donde se guarda lo que haces

**para cambiar parámetros:**
en el archivo de configuracion:



En el bloque optimizer del train_config se define la tasa de aprendizaje (learning_rate) y el momentum. Por ejemplo

Si quieres ajustar la tasa de aprendizaje, cambia learning_rate_base y/o warmup_learning_rate.
Si deseas modificar el momentum, cambia momentum_optimizer_value.

Número de clases (num_classes):
Aunque tengas un archivo label_map.pbtxt que mapea los IDs a nombres, el modelo necesita saber cuántas salidas generar en su capa de clasificación. En el bloque model { ssd { ... } } busca la línea:
cambialo al numero de clases tuya

calcular las epochs:

num_steps = (número de imágenes de entrenamiento / batch_size) * epochs

para navegar al directorio y correr entrenamiento


In [ ]:
%cd /content/models/research/object_detection

/content/models/research/object_detection


In [ ]:
%cd /content/models/research/object_detection
!python model_main_tf2.py \
    --pipeline_config_path=/content/models/research/object_detection/configs/tf2/ssd_efficientdet_d1_640x640_coco17_tpu-8.config \
    --model_dir=/content/efficientdet_model \
    --alsologtostderr

/content/models/research/object_detection
2025-03-17 09:07:42.677970: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742202462.710410    2819 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742202462.720192    2819 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-17 09:07:42.765182: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Traceback (most recent call last):
  File "/content/models/research/object_

# validar:

In [ ]:
!python main.py \
  --mode=eval \
  --validation_file_pattern="/content/dataset/tf_records/val-*.tfrecord" \
  --model_name="efficientdet-d0" \
  --model_dir="/content/model_dir/d0" \
  --eval_samples=2000



Inferencia

In [ ]:
!python model_inspect.py \
  --mode=export \
  --model_name="efficientdet-d0" \
  --model_dir="/content/model_dir/d0" \
  --saved_model_dir="/content/savedmodel" \
  --tflite_path="/content/model.tflite" \
  --export_format="tflite"

PROBANDO COSAS NUEVAS
